In [1]:
# pip install google-cloud-bigquery google-cloud-storage google-cloud-aiplatform


In [2]:
import pandas as pd
import db_dtypes
from google.cloud import bigquery,aiplatform,storage
project_id = "industry-buying"
bigquery_client = bigquery.Client(project=project_id)
storage_client = storage.Client()
from google.cloud import aiplatform
client = bigquery.Client(project=project_id)
from datetime import datetime

In [3]:
# #  BigQuery table name
# table_name = "batch_input_cm"
# project_id = "industry-buying"
# dataset_name= "reports"



In [4]:
# Initialize Vertex AI model service client
aiplatform.init(project=project_id)

In [5]:
# Initialize connection
aiplatform.init(location='us-central1')

In [6]:
# Get model that will make a batch prediction
model_id = 'projects/industry-buying/locations/us-central1/models/1077334478243758080'
model = aiplatform.Model(model_id)

In [7]:
    #refreshing input table for batch data

    tablequery= '''


    create or replace table google_clid.batchinput_cm as 

    SELECT
      CAST(COALESCE(b.length,0) AS float64) AS length,
      CAST(COALESCE(b.breadth,0) AS float64) AS breadth,
      CAST(COALESCE(b.height,0) AS float64) AS height,
      CAST(COALESCE(b.weight,0) AS float64) AS weight,
      order_date,
      COALESCE(booked_device,'android') AS booked_device,
      order_quantity,
      CAST(order_tax AS float64) order_tax,
      CAST(coupoun_discount AS float64) AS coupoun_discount,
      CAST(a.total AS float64) total,
      a.sku,
      COALESCE(sku_type,'MarketPlace SKU') AS sku_type,
      COALESCE(Brand,'lazer') AS Brand,
      COALESCE(leaf_category,'Air Cooler') AS leaf_category,
      COALESCE(category,'Appliances') AS category,
      COALESCE(c1_category,'Furniture') AS c1_category,
      payment_mode,
      a.line_status AS state,
      city,
    IF
      (LENGTH(COALESCE(utm_source, 'unknown')) < 4, 'unknown', COALESCE(utm_source, 'unknown')) AS utm_source,
      CAST(shipping_charge AS float64) AS shipping_charge,
      CAST(cod_charge AS float64) cod_charge,
      CAST(bulk_discount AS float64) AS bulk_discount,
    IF
      (LENGTH(COALESCE(utm_campaign, 'unknown')) < 4, 'unknown', COALESCE(utm_campaign, 'unknown')) AS utm_campaign,
    IF
      (LENGTH(COALESCE(utm_medium, 'unknown')) < 4, 'unknown', COALESCE(utm_medium, 'unknown')) AS utm_medium,
      CAST(pincode AS int64) pincode,
      CAST(order_no AS int64) order_no,
      COALESCE(allocated_vendor_code, 0)allocated_vendor_code,
      COALESCE(warehouse_code, 'unknown') warehouse_code,
      CASE
        WHEN Order_Check_Gstin= "YES" THEN TRUE
      ELSE
      FALSE
    END
      AS Order_Check_Gstin,
      CAST(B_GMV AS float64) AS B_GMV,
      CAST(tax_excl_incl_ship_cod AS float64) tax_excl_incl_ship_cod,
      CAST(total_cp_actual AS float64) total_cp_actual,
      a.User_AON,
      CAST( (tax_excl_incl_ship_cod - total_cp_actual) / tax_excl_incl_ship_cod AS float64) AS GP_PCT,
      CAST(( tax_excl_incl_ship_cod - total_cp_actual ) AS float64) AS GP,
      CAST((((CASE
                WHEN return_flag = 0 THEN tax_excl_incl_ship_cod - total_cp_actual
              ELSE
              0
            END
              )- freight_total)) AS float64) AS CM
    FROM
      `analytics_writedb.ff_daily_cm1_tbl` a
    LEFT JOIN
      reports.sku_dimension b
    ON
      a.sku=b.sku
    WHERE
      total_cp_actual<=tax_excl_incl_ship_cod
      AND order_date between DATE(DATE_SUB(CURRENT_DATE(), INTERVAL 32 day)) and   DATE(DATE_SUB(CURRENT_DATE(), INTERVAL 1 day))
      AND tax_excl_incl_ship_cod>0 



    '''


    client.query(tablequery)

QueryJob<project=industry-buying, location=US, id=b8635152-0897-42e1-8266-953957a8ab1b>

In [8]:
# defining batch input, output and job name 


current_date = datetime.now().strftime('%d%m%Y')
batch_current_date = f'batch_{current_date}'  

job_display_name = batch_current_date
bigquery_source = 'bq://industry-buying.google_clid.batchinput_cm'
bigquery_destination_prefix = 'bq://industry-buying.google_clid'


In [9]:
# Creating batch prediction job
batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    bigquery_source=bigquery_source,
    bigquery_destination_prefix=bigquery_destination_prefix)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/378362668644/locations/us-central1/batchPredictionJobs/3973545068987416576
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/378362668644/locations/us-central1/batchPredictionJobs/3973545068987416576')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/3973545068987416576?project=378362668644
BatchPredictionJob projects/378362668644/locations/us-central1/batchPredictionJobs/3973545068987416576 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/378362668644/locations/us-central1/batchPredictionJobs/3973545068987416576 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/378362668644/locations/us-central1/batchPredictionJobs/3973545068987416576 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/378362668644/locations/us-central1/batchPredictionJobs/

In [10]:
# job output information
batch_prediction_job.output_info

bigquery_output_dataset: "bq://industry-buying.google_clid"
bigquery_output_table: "predictions_2023_12_05T20_03_30_720Z_601"

In [11]:
# Convert output job information into string
output_info_str = str(batch_prediction_job.output_info)
output_info_str

'bigquery_output_dataset: "bq://industry-buying.google_clid"\nbigquery_output_table: "predictions_2023_12_05T20_03_30_720Z_601"\n'

In [12]:
# Split elements of the output by whitespace
output_info_list = output_info_str.split()
output_info_list

['bigquery_output_dataset:',
 '"bq://industry-buying.google_clid"',
 'bigquery_output_table:',
 '"predictions_2023_12_05T20_03_30_720Z_601"']

In [13]:
output_info_list = [x.replace('"', '') for x in output_info_list]
output_info_list

['bigquery_output_dataset:',
 'bq://industry-buying.google_clid',
 'bigquery_output_table:',
 'predictions_2023_12_05T20_03_30_720Z_601']

In [14]:
# Part which holds project and dataset name
dataset_str = output_info_list[1]

# Part which holds table name
table_str = output_info_list[3]

print(f'Project & dataset: {dataset_str} \nTable: {table_str}')

Project & dataset: bq://industry-buying.google_clid 
Table: predictions_2023_12_05T20_03_30_720Z_601


In [15]:
# Get everything after 'bg://' part from the dataset_str
import re

# Define regex pattern to look for
pattern = r'(?<=bq://).*'

# Run the regex search and show variable
result = re.search(pattern, dataset_str)
dataset_str = result.group(0)
dataset_str

'industry-buying.google_clid'

In [16]:
# Define Table ID 
table_id = dataset_str + '.' + table_str
table_id

'industry-buying.google_clid.predictions_2023_12_05T20_03_30_720Z_601'

In [17]:
# Read bigquery library and define client
from google.cloud import bigquery
client = bigquery.Client()

In [18]:
# Define query which will create a table in BigQuery and run it
query = f"""


CREATE OR REPLACE TABLE google_clid.cm_prediction_daily AS
(


select utm_campaign,predicted_CM.value as predicted_cm , order_no 
   
  FROM 
  
  {table_id}

)
"""
client.query(query)

QueryJob<project=industry-buying, location=US, id=f033fb1b-861f-4073-b78d-199101f43063>

In [19]:
# # Delete original raw table with results as it is no longer needed
# client.delete_table(table_id)
# print("Deleted table '{}'.".format(table_id))

In [20]:
query2= """

create or replace view  google_clid.readjustment_conversion as 
select  
  Google_Click_ID, 
   a.utm_campaign as Conversion_Name, 
   Conversion_Time, 
   Adjustment_Time,
   'RESTATE' as AdjustmentType,

  sum(case when predicted_cm<=1 then 1 else floor(predicted_cm) end) as Adjusted_Value  ,
 
 
 'INR' as Adjusted_Value_Currency 


 from 

(select utm_campaign,predicted_cm as predicted_cm , order_no from google_clid.cm_prediction_daily ) a 

inner join 

(select  a.gclid as Google_Click_ID


,a.order_no, concat(left(cast(timestamp(a.order_booked_date) as string),19),'+0000') Conversion_Time,

concat(left(cast(timestamp(current_datetime()) as string),19),'+0000') Adjustment_Time


 
 
 from `emtex.orders_order`  a


 where 


 length(gclid)>5 
 ) b 

on safe_cast(a.order_no as int64)=safe_cast(b.order_no as int64)


group by 1,2,3,4





"""


client.query(query2)




QueryJob<project=industry-buying, location=US, id=2f32c7a1-f9e4-4797-93f6-95d0b959dc3c>

In [21]:

query3= """

select * from google_clid.readjustment_conversion

"""
client.query(query3)

QueryJob<project=industry-buying, location=US, id=43f7ac7b-bce9-417e-a5b3-6bb83d25b645>

In [22]:

#getting this query3 data into dataframe to export this as a csv
df = client.query(query3).to_dataframe()

In [23]:
file_path= "D:\\Prediction\\New_Working_21112023\\clid\\new_clid.csv"
df.to_csv(file_path, index=False)



In [24]:
query4= """select * from google_clid.readjustment_conversion"""


In [25]:
#  BigQuery table name
table_name = "glcid_log_table"
project_id = "industry-buying"
dataset_name= "google_clid"



In [26]:
destination_table = f"{project_id}.{dataset_name}.{table_name}"
query_job = client.query(query4)
# Get the query results
results = query_job.result()

# Insert the results into the destination table
rows_to_insert = [dict(row) for row in results]

client.insert_rows_json(destination_table, rows_to_insert)



[]